In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, roc_auc_score,accuracy_score
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import cross_val_score
import eli5
import ast
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.feature_selection import f_classif
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import precision_recall_curve
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib.colors import ListedColormap

In [3]:
piRNA_rf=pd.read_csv("R/GEO_count/pan_train_new.csv")

In [ ]:
piRNA_rf=piRNA_rf.set_index('Unnamed: 0')

In [ ]:
X_train = piRNA_rf.drop(columns=['group'])
y_train = piRNA_rf['group']

In [ ]:
test_logit=pd.read_csv("R/GEO_count/pan_test_new.csv")
test_logit=pd.read_csv("R/GEO_count/tissue_test_new.csv")
test_logit=pd.read_csv("R/GEO_count/plasma_test.csv")
test_logit=pd.read_csv("R/GEO_count/exosome_test_new.csv")
test_logit=pd.read_csv("R/GEO_count/pan_test_new_CH.csv")
test_logit=pd.read_csv("R/GEO_count/exosome_test_new_BC.csv")
test_logit=pd.read_csv("R/GEO_count/GSE83527_rf2.csv")
test_logit=pd.read_csv("R/GEO_count/GSE62182_rf2.csv")

In [ ]:
test_logit=test_logit.set_index('Unnamed: 0')
X_test=test_logit.drop(columns=['group'])
y_test = test_logit['group']

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred_log = model.predict(X_test)

# Compute accuracy for validation and test sets, threshold 0.5
acc_log = accuracy_score(y_test, y_pred_log)

# Compute predicted probabilities for AUC calculation
y_prob_log = model.predict_proba(X_test)[:, 1]

# Compute AUC for validation and test sets
auc_log = roc_auc_score(y_test, y_prob_log)

print(f"Accuracy: {acc_log:.4f}")
print(f"AUC: {auc_log:.4f}")

#roc_curve
y_binary_log = label_binarize(y_test, classes=['normal', 'tumor']).flatten()
fpr_log, tpr_log, thresholds_test = roc_curve(y_binary_log, y_prob_log)
youden_J_test = tpr_log - fpr_log
# Locate the index of the largest J statistic
ix_test = np.argmax(youden_J_test)
optimal_threshold_test = thresholds_test[ix_test]

print('Best Threshold=%f, Youden J=%.3f' % (optimal_threshold_test, youden_J_test[ix_test]))

In [ ]:
y_pred_train = model.predict(X_train)

# Compute accuracy for validation and test sets, threshold 0.5
acc_train = accuracy_score(y_train, y_pred_train)

# Compute predicted probabilities for AUC calculation
y_prob_train = model.predict_proba(X_train)[:, 1]

# Compute AUC for validation and test sets
auc_train = roc_auc_score(y_train, y_prob_train)

print(f"Accuracy: {acc_train:.4f}")
print(f"AUC: {auc_train:.4f}")

#roc_curve
y_binary_train = label_binarize(y_train, classes=['normal', 'tumor']).flatten()
fpr_train, tpr_train, thresholds_train = roc_curve(y_binary_train, y_prob_train)
youden_J_train = tpr_train - fpr_train
# Locate the index of the largest J statistic
ix_train = np.argmax(youden_J_train)
optimal_threshold_train = thresholds_train[ix_train]

print('Best Threshold=%f, Youden J=%.3f' % (optimal_threshold_train, youden_J_train[ix_train]))

precision_train, recall_train, thresholds_train = precision_recall_curve(y_train, y_prob_train,pos_label='tumor')
auprc_train = auc(recall_train, precision_train)
print("auprc_train:", auprc_train)

In [ ]:
f1_scores_train = 2 * (precision_train * recall_train) / (precision_train + recall_train)
# Handle division by zero in case precision and recall are both zero
f1_scores_train = np.nan_to_num(f1_scores_train)

optimal_idx_train = np.argmax(f1_scores_train)
optimal_threshold_train = thresholds_train[optimal_idx_train]
best_f1_score_train = f1_scores_train[optimal_idx_train] 
print("Optimal threshold:", optimal_threshold_train)
print("best f score:", best_f1_score_train)


y_pred_optimal_threshold_train = np.where(y_prob_train >= optimal_threshold_train, "tumor", "normal")

tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train, y_pred_optimal_threshold_train).ravel()

ppv_train = tp_train / (tp_train + fp_train) if (tp_train + fp_train) != 0 else 0

# Negative Predictive Value
npv_train = tn_train / (tn_train + fn_train) if (tn_train + fn_train) != 0 else 0

# Sensitivity (Recall)
sensitivity_train = tp_train / (tp_train + fn_train) if (tp_train + fn_train) != 0 else 0

# Specificity
specificity_train = tn_train / (tn_train + fp_train) if (tn_train + fp_train) != 0 else 0

print(f"PPV/Precision: {ppv_train}")
print(f"NPV: {npv_train}")
print(f"Sensitivity/Recall: {sensitivity_train}")
print(f"Specificity: {specificity_train}")

In [ ]:
precision_test, recall_test, thresholds_test = precision_recall_curve(y_test, y_prob_log,pos_label='tumor')
f1_scores_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
# Handle division by zero in case precision and recall are both zero
f1_scores_test = np.nan_to_num(f1_scores_test)

optimal_idx_test = np.argmax(f1_scores_test)
optimal_threshold_test = thresholds_test[optimal_idx_test]
best_f1_score_test = f1_scores_test[optimal_idx_test] 
print("Optimal threshold:", optimal_threshold_test)
print("best f score:", best_f1_score_test)


y_pred_optimal_threshold_test = np.where(y_prob_log >= optimal_threshold_test, "tumor", "normal")

tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_optimal_threshold_test).ravel()

ppv_test = tp_test / (tp_test + fp_test) if (tp_test + fp_test) != 0 else 0

# Negative Predictive Value
npv_test = tn_test / (tn_test + fn_test) if (tn_test + fn_test) != 0 else 0

# Sensitivity (Recall)
sensitivity_test = tp_test / (tp_test + fn_test) if (tp_test + fn_test) != 0 else 0

# Specificity
specificity_test = tn_test / (tn_test + fp_test) if (tn_test + fp_test) != 0 else 0


#AUPRC
auprc_test = auc(recall_test, precision_test)
print("auprc_test:", auprc_test)


print(f"PPV/Precision: {ppv_test}")
print(f"NPV: {npv_test}")
print(f"Sensitivity/Recall: {sensitivity_test}")
print(f"Specificity: {specificity_test}")

In [ ]:
# score

plt.figure(figsize=(5, 5))

# plot ROC curve for the train set
plt.plot(fpr_train, tpr_train, label=f'train_AUC : {auc_train:.2f}')

# plot ROC curve for the train set
plt.plot(fpr_log, tpr_log, label=f'test_AUC : {auc_log:.2f}')

# Plot diagonal line
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')

# Labels, title, and other settings
plt.xlabel('1 - Specificity (False Positive Rate)', fontweight='bold')
plt.ylabel('Sensitivity (True Positive Rate)', fontweight='bold')
legend = plt.legend(loc='lower right')
plt.grid(False)
plt.show()

In [ ]:
precision_train, recall_train, _ = precision_recall_curve(y_train, y_prob_train,pos_label='tumor')
precision_test, recall_test, _ = precision_recall_curve(y_test, y_prob_log,pos_label='tumor')
# precision_val, recall_val, _ = precision_recall_curve(y_val, y_pred_proba_val,pos_label='tumor')


plt.figure(figsize=(5, 5))
plt.plot(recall_train, precision_train, marker='.', label=f'Training Set AUCPRC = {auprc_train:.2f}', color='blue')

# Plot for test set
# Plot for test set
plt.plot(recall_test, precision_test, marker='.', label=f'Test Set AUCPRC = {auprc_test:.2f}', color='green')

# Plot for validation set
# plt.plot(recall_val, precision_val, marker='.', label='Validation', color='red')

# Labeling the axes and setting the title
plt.xlabel('Recall',fontweight='bold')
plt.ylabel('Precision',fontweight='bold')

# Display the legend
plt.legend()
# Show the plot
plt.show()

In [ ]:
def compute_metrics_from_cm(cm):
    # Extracting values from the confusion matrix
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    TP = cm[1, 1]

    # Calculating Sensitivity and Specificity
    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)

    return sensitivity, specificity

In [ ]:
y_pred_test_labels = ["tumor" if p >= 0.34 else "normal" for p in y_prob_log]

In [ ]:
cm = confusion_matrix(y_test, y_pred_test_labels)
print("Confusion Matrix:")
print(cm)
sensitivity_test, specificity_test = compute_metrics_from_cm(cm)
print("sensitivity_test, specificity_test:")
print(sensitivity_test, specificity_test)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(y_true, y_pred, classes=None, figsize=(7, 5), text_size=14):
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    n_classes = cm.shape[0]

    if classes is None:
        classes = [str(i) for i in range(n_classes)]
        
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.matshow(cm_norm, cmap=plt.cm.Blues)
    fig.colorbar(cax)

    ax.set(title='stage I&II GSE83527', 
           xlabel='Predicted Label', 
           ylabel='True Label', 
           xticks=np.arange(n_classes), 
           yticks=np.arange(n_classes), 
           xticklabels=classes, 
           yticklabels=classes)
    ax.xaxis.set_label_position('bottom')
    ax.xaxis.tick_bottom()

    ax.yaxis.label.set_size(text_size)
    ax.xaxis.label.set_size(text_size)
    ax.title.set_size(text_size * 1.2)
    
    threshold = (cm_norm.max() + cm_norm.min()) / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, f'{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)', 
                 horizontalalignment='center', 
                 color='white' if cm_norm[i, j] > threshold else 'black', 
                 size=text_size)

In [ ]:
classes = ['Normal', 'Tumor']
plot_confusion_matrix(y_test, y_pred_test_labels, classes)
plt.show(